In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import StaleElementReferenceException
import mysql.connector
import time
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException, WebDriverException

# MySQL database connection setup

In [3]:
def connect_to_database():
    connection = mysql.connector.connect(
        host='localhost',
        user='root',
        password='Pavan@22',
        database='redbus_data'
    )
    return connection

# inserting data into MySQL
def insert_into_database(data, connection):
    cursor = connection.cursor()
    insert_query = """
    INSERT INTO bus_routes (route_name, route_link, busname, bustype, departing_time, duration, reaching_time, star_rating, price, seats_available)
    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
    """
    cursor.execute(insert_query, data)
    connection.commit()


# Scrapping

In [4]:
def safe_click(element):
    retry_count = 3
    for _ in range(retry_count):
        try:
            element.click()
            return True
        except StaleElementReferenceException:
            time.sleep(1)
    return False
    
def extract_bus_details(route_name, route_link, connection):
    """Extract detailed bus information from the current page."""
    try:
        # Wait for bus items
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'bus-item')] | //div[contains(@class, 'error-view oops-page')]"))
        )

        # Locate all bus items
        bus_items = driver.find_elements(By.XPATH, "//div[contains(@class, 'bus-item')]")

        # If no bus items are found, navigate back to the routes page
        if not bus_items:
            print(f"No bus items found for route: {route_name}. Returning to the routes page.")
            driver.back()  # Return to the routes page
            WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[class='route']")))
            time.sleep(3)
            return

        # Proceed with extraction if bus items are found
        print(f"Found {len(bus_items)} buses for route: {route_name}.")

        # Loop through each bus detail container
        for bus_item in bus_items:
            try:
                bus_name = bus_item.find_element(By.XPATH, ".//div[contains(@class, 'travels lh-24 f-bold d-color')]").text
                bus_type = bus_item.find_element(By.XPATH, ".//div[contains(@class, 'bus-type')]").text
                departure_time = bus_item.find_element(By.XPATH, ".//div[contains(@class, 'dp-time')]").text
                arrival_time = bus_item.find_element(By.XPATH, ".//div[contains(@class, 'bp-time')]").text
                duration = bus_item.find_element(By.XPATH, ".//div[contains(@class, 'dur l-color lh-24')]").text

                star_rating = None
                try:
                    star_rating_element = bus_item.find_element(By.XPATH, ".//div[contains(@class, 'rating-sec lh-24')]/div[contains(@class, 'lh-18 rating')]/span")
                    star_rating = float(star_rating_element.text) if star_rating_element.text else 0.0
                except NoSuchElementException:
                    star_rating = 0.0

                fare = bus_item.find_element(By.XPATH, "//div[@class='fare d-block']/span[@class='f-19 f-bold']").text
                seats_available = bus_item.find_element(By.XPATH, ".//div[contains(@class, 'seat-left')]").text

                fare = float(fare.replace(",", ""))  # Handle comma in fare
                seats_available = int(seats_available.split()[0]) if seats_available else 0

                bus_data = (
                    route_name, route_link,
                    bus_name, bus_type,
                    departure_time, duration,
                    arrival_time, star_rating,
                    fare, seats_available
                )

                # Insert data into the database
                insert_into_database(bus_data, connection)

            except Exception as e:
                print(f"Error processing bus item: {e}")
                continue

        print(f"Inserted {len(bus_items)} records for route: {route_name}")

    except TimeoutException:
        # If no bus details are loaded within the wait time, go back to the routes page
        print(f"No bus details found for route: {route_name}. Returning to the routes page.")
        driver.back()
        WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[class='route']")))
        time.sleep(3)

    except Exception as e:
        print(f"Error extracting bus details for route: {route_name}: {e}")
        driver.back()
        WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[class='route']")))
        time.sleep(3)


def navigate_route_pages(connection):
    """Navigate through all route pages and process routes."""
    while True:
        try:
            # Locate the pagination controls
            page_numbers = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_pageTabs')]")
            print(f"Found {len(page_numbers)} page numbers.")


            if not page_numbers:
                print("No page numbers found. Exiting pagination loop.")
                break

            for index in range(len(page_numbers)):
                try:
                    # Re-fetch page numbers to avoid stale references
                    page_numbers = driver.find_elements(By.XPATH, "//div[contains(@class, 'DC_117_pageTabs')]")
                    page_number = page_numbers[index]

                    print(f"Navigating to page number {index + 1}")

                    driver.execute_script("arguments[0].scrollIntoView(true);", page_number)
                    time.sleep(2)

                    # Click on the page number to navigate
                    if safe_click(page_number):
                        time.sleep(3)
                        process_routes_on_page(connection)

                except (TimeoutException, StaleElementReferenceException, WebDriverException) as e:
                    print(f"Error navigating to page number {index + 1}: {e}")
                    continue

            break

        except Exception as e:
            print(f"Error navigating route pages: {e}")
            break


def process_routes_on_page(connection):
    """Process all routes on the current page."""
    routes = driver.find_elements(By.CSS_SELECTOR, "a[class='route']")
    print(f"Found {len(routes)} routes on current page.")

    for index in range(len(routes)):
        try:
            # Re-fetch the route list to avoid stale elements
            routes = driver.find_elements(By.CSS_SELECTOR, "a[class='route']")
            route = routes[index]

            route_name = route.text
            route_link = route.get_attribute('href')

            driver.execute_script("arguments[0].scrollIntoView(true);", route)
            time.sleep(2)

            # Click the route link
            if safe_click(route):
                time.sleep(3)
                WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.XPATH, "//div[contains(@class, 'bus-item')]")))

                # Extract bus details
                
                click_view_buses_buttons()
                
                scroll_and_load_all_buses()
                extract_bus_details(route_name, route_link, connection)

                # Go back to the previous page
                driver.back()
                WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[class='route']")))
                time.sleep(3)

        except (TimeoutException, StaleElementReferenceException, WebDriverException) as e:
            print(f"Error processing route: {e}")
            driver.back()
            WebDriverWait(driver, 20).until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "a[class='route']")))
            time.sleep(3)



def scroll_and_load_all_buses():
    """Scroll to the end of the page to load all dynamic content."""
    last_height = driver.execute_script("return document.body.scrollHeight")
    
    while True:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(3)  # Wait for new content to load
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    print("All buses loaded successfully.")
    
    
# Function to click all 'View Buses' buttons on a page
def click_view_buses_buttons():
    """Click on all 'View Buses' buttons."""
    try:
        view_buses_buttons = driver.find_elements(By.XPATH, "//div[contains(@class, 'button') and contains(text(), 'View Buses')]")
        print(f"Found {len(view_buses_buttons)} 'View Buses' buttons.")
        for button in view_buses_buttons:
            try:
                driver.execute_script("arguments[0].scrollIntoView(true);", button)
                time.sleep(2)
                driver.execute_script("arguments[0].click();", button)
                time.sleep(3)
            except (TimeoutException, StaleElementReferenceException, WebDriverException) as e:
                print(f"Error clicking 'View Buses' button: {e}")
                continue
    except Exception as e:
        print(f"Error in 'View Buses' button processing: {e}")



def process_routes_for_card(rtc_card, connection):
    """Process all routes for a given RTC card."""
    try:
        rtc_card_name = rtc_card.find_element(By.CSS_SELECTOR, "div.rtcName").text  # Get the RTC card name
        print(f"Processing RTC card: {rtc_card_name}")  # Print the RTC card being processed

        driver.execute_script("arguments[0].scrollIntoView(true);", rtc_card)
        time.sleep(2)
        driver.execute_script("arguments[0].click();", rtc_card)
        time.sleep(3)
        navigate_route_pages(connection)
    except Exception as e:
        print(f"Error processing RTC card: {e}")
    

# 1. APSRTC

In [41]:
try:

    driver = webdriver.Chrome()

    rtc_card_link = 'https://www.redbus.in/online-booking/apsrtc/?utm_source=rtchometile'
    driver.get(rtc_card_link)

    # Wait until the routes are loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='route']")))

    # Connect to the MySQL database
    connection = connect_to_database()

    navigate_route_pages(connection)

finally:
    driver.quit()
    connection.close()
    print("Finished processing all data and closed the database connection.")


Table 'bus_routes' has been truncated.
Found 5 page numbers.
Navigating to page number 1
Found 10 routes on current page.
Found 2 'View Buses' buttons.
All buses loaded successfully.
Found 1258 buses for route: Hyderabad to Vijayawada.
Inserted 1258 records for route: Hyderabad to Vijayawada
Error processing route: Message: 

Found 2 'View Buses' buttons.
All buses loaded successfully.
Found 892 buses for route: Vijayawada to Hyderabad.
Inserted 892 records for route: Vijayawada to Hyderabad
Error processing route: Message: 

Found 2 'View Buses' buttons.
All buses loaded successfully.
Found 384 buses for route: Hyderabad to Ongole.
Inserted 384 records for route: Hyderabad to Ongole
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 168 buses for route: Kakinada to Visakhapatnam.
Inserted 168 records for route: Kakinada to Visakhapatnam
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Fou

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 102 buses for route: Tirupati to Chennai.
Inserted 102 records for route: Tirupati to Chennai
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 476 buses for route: Vijayawada to Visakhapatnam.
Inserted 476 records for route: Vijayawada to Visakhapatnam
Error processing route: Message: 

Navigating to page number 5
Found 6 routes on current page.
Found 2 'View Buses' buttons.
All buses loaded successfully.
Found 68 buses for route: Hyderabad to Macherla (andhra pradesh).
Inserted 68 records for route: Hyderabad to Macherla (andhra pradesh)
Error processing route: Message: 

Found 2 'View Buses' buttons.
All buses loaded successfully.
Found 30 buses for route: Hyderabad to Rajahmundry.
Error processing bus item: could not convert string to float: ''
Error processing bus item: could not convert string to float: ''
Error processing bus item: could not convert string to fl

Error navigating to page number 5: Message: 

Finished processing all data and closed the database connection.


# 2.KSRTC

In [42]:
try:

    driver = webdriver.Chrome()

    rtc_card_link = 'https://www.redbus.in/online-booking/ksrtc-kerala/?utm_source=rtchometile'
    driver.get(rtc_card_link)

    # Wait until the routes are loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='route']")))

    # Connect to the MySQL database
    connection = connect_to_database()

    navigate_route_pages(connection)

finally:
    driver.quit()
    connection.close()
    print("Finished processing all data and closed the database connection.")


Found 2 page numbers.
Navigating to page number 1
Found 10 routes on current page.
Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 68 buses for route: Bangalore to Kozhikode.
Inserted 68 records for route: Bangalore to Kozhikode
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 98 buses for route: Kozhikode to Bangalore.
Inserted 98 records for route: Kozhikode to Bangalore
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 180 buses for route: Kozhikode to Ernakulam.
Inserted 180 records for route: Kozhikode to Ernakulam
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 180 buses for route: Ernakulam to Kozhikode.
Inserted 180 records for route: Ernakulam to Kozhikode
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 42 buses for route: Bangalore to Kannur.
Inser

# 3. TSRTC

In [2]:
try:

    driver = webdriver.Chrome()

    rtc_card_link = 'https://www.redbus.in/online-booking/tsrtc/?utm_source=rtchometile'
    driver.get(rtc_card_link)

    # Wait until the routes are loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='route']")))

    # Connect to the MySQL database
    connection = connect_to_database()
    
    navigate_route_pages(connection)

finally:
    driver.quit()
    connection.close()
    print("Finished processing all data and closed the database connection.")


Found 3 page numbers.
Navigating to page number 1
Found 10 routes on current page.
Found 2 'View Buses' buttons.
All buses loaded successfully.
Found 1094 buses for route: Hyderabad to Vijayawada.
Inserted 1094 records for route: Hyderabad to Vijayawada
Error processing route: Message: 

Found 2 'View Buses' buttons.
All buses loaded successfully.
Found 144 buses for route: Khammam to Hyderabad.
Inserted 144 records for route: Khammam to Hyderabad
Error processing route: Message: 

Found 2 'View Buses' buttons.
All buses loaded successfully.
Found 228 buses for route: Hyderabad to Khammam.
Inserted 228 records for route: Hyderabad to Khammam
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 46 buses for route: Hyderabad to Srisailam.
Inserted 46 records for route: Hyderabad to Srisailam
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 94 buses for route: Karimnagar to Hyderabad.
Ins

# 4.KTCL

In [8]:
try:

    driver = webdriver.Chrome()

    rtc_card_link = 'https://www.redbus.in/online-booking/ktcl/?utm_source=rtchometile'
    driver.get(rtc_card_link)

    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='route']")))

    # Connect to the MySQL database
    connection = connect_to_database()
    
    navigate_route_pages(connection)

finally:
    driver.quit()
    connection.close()
    print("Finished processing all data and closed the database connection.")


Found 4 page numbers.
Navigating to page number 1
Found 10 routes on current page.
Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 126 buses for route: Pune to Goa.
Inserted 126 records for route: Pune to Goa
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 150 buses for route: Goa to Pune.
Inserted 150 records for route: Goa to Pune
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 106 buses for route: Mumbai to Goa.
Inserted 106 records for route: Mumbai to Goa
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 50 buses for route: Bangalore to Goa.
Inserted 50 records for route: Bangalore to Goa
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 40 buses for route: Goa to Bangalore.
Inserted 40 records for route: Goa to Bangalore
Error processing route: Message:

# 5.RSTC 

In [9]:
try:
    driver = webdriver.Chrome()

    rtc_card_link = 'https://www.redbus.in/online-booking/rsrtc/?utm_source=rtchometile'
    driver.get(rtc_card_link)

    # Wait until the routes are loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='route']")))

    # Connect to the MySQL database
    connection = connect_to_database()

    navigate_route_pages(connection)

finally:
    driver.quit()
    connection.close()
    print("Finished processing all data and closed the database connection.")


Found 2 page numbers.
Navigating to page number 1
Found 10 routes on current page.
Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 64 buses for route: Udaipur to Jodhpur.
Inserted 64 records for route: Udaipur to Jodhpur
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 194 buses for route: Jodhpur to Ajmer.
Inserted 194 records for route: Jodhpur to Ajmer
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 206 buses for route: Beawar (Rajasthan) to Jaipur (Rajasthan).
Inserted 206 records for route: Beawar (Rajasthan) to Jaipur (Rajasthan)
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 228 buses for route: Sikar to Jaipur (Rajasthan).
Inserted 228 records for route: Sikar to Jaipur (Rajasthan)
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 110 buses for rout

# 6.sbstc

In [7]:
try:
    driver = webdriver.Chrome()

    rtc_card_link = 'https://www.redbus.in/online-booking/south-bengal-state-transport-corporation-sbstc/?utm_source=rtchometile'
    driver.get(rtc_card_link)

    # Wait until the routes are loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='route']")))

    # Connect to the MySQL database
    connection = connect_to_database()

    navigate_route_pages(connection)

finally:
    driver.quit()
    connection.close()
    print("Finished processing all data and closed the database connection.")


Found 5 page numbers.
Navigating to page number 1
Found 10 routes on current page.
Found 2 'View Buses' buttons.
All buses loaded successfully.
Found 220 buses for route: Durgapur to Calcutta.
Inserted 220 records for route: Durgapur to Calcutta
Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 8 buses for route: Kolkata to Burdwan.
Inserted 8 records for route: Kolkata to Burdwan
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 60 buses for route: Haldia to Calcutta.
Inserted 60 records for route: Haldia to Calcutta
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 62 buses for route: Kolkata to Haldia.
Inserted 62 records for route: Kolkata to Haldia
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 2 buses for route: Kolkata to Durgapur (West Bengal).
Error processing bus item: Message: no such element: Unable t

All buses loaded successfully.
Found 2 buses for route: Kolkata to Chandipur (West Bengal).
Inserted 2 records for route: Kolkata to Chandipur (West Bengal)
Error processing route: Message: 

Error processing route: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF685559412+29090]
	(No symbol) [0x00007FF6854CE239]
	(No symbol) [0x00007FF68538B1DA]
	(No symbol) [0x00007FF6853DEFE7]
	(No symbol) [0x00007FF6853DF23C]
	(No symbol) [0x00007FF6854297C7]
	(No symbol) [0x00007FF68540672F]
	(No symbol) [0x00007FF6854265A2]
	(No symbol) [0x00007FF685406493]
	(No symbol) [0x00007FF6853D09D1]
	(No symbol) [0x00007FF6853D1B31]
	GetHandleVerifier [0x00007FF68587871D+3302573]
	GetHandleVerifier [0x00007FF6858C4243+3612627]
	GetHandleVerifier [0x00007FF6858BA417+3572135]
	GetHandleVerifier [0x00007FF685615EB6+801862]
	(No symbol) [0x00007FF6854D945F]
	(No symbol) [0x00007FF6854D4FB4]
	(No symbol) [0x00007FF6854D5140]
	(No symbol) [0x00007FF6854C461F]
	BaseThreadInitThunk [0x00007FFAF3D4257D+29]
	Rtl

# 7.HRTC

In [5]:
try:
    driver = webdriver.Chrome()

    rtc_card_link = 'https://www.redbus.in/online-booking/hrtc/?utm_source=rtchometile'
    driver.get(rtc_card_link)

    # Wait until the routes are loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='route']")))

    # Connect to the MySQL database
    connection = connect_to_database()

    navigate_route_pages(connection)

finally:
    driver.quit()
    connection.close()
    print("Finished processing all data and closed the database connection.")


Found 4 page numbers.
Navigating to page number 1
Found 10 routes on current page.
Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 40 buses for route: Delhi to Shimla.
Inserted 40 records for route: Delhi to Shimla
Error processing route: Message: 

Found 2 'View Buses' buttons.
All buses loaded successfully.
Found 28 buses for route: Chandigarh to Hamirpur (Himachal Pradesh).
Inserted 28 records for route: Chandigarh to Hamirpur (Himachal Pradesh)
Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 10 buses for route: Hamirpur (Himachal Pradesh) to Chandigarh.
Inserted 10 records for route: Hamirpur (Himachal Pradesh) to Chandigarh
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 34 buses for route: Shimla to Delhi.
Inserted 34 records for route: Shimla to Delhi
Error processing route: Message: 

Found 2 'View Buses' buttons.
All buses loaded successfully.
Found 318 buses for route: Delhi to Chandig

# 8.Astc

In [10]:
try:

    driver = webdriver.Chrome()

    rtc_card_link = 'https://www.redbus.in/online-booking/astc/?utm_source=rtchometile'
    driver.get(rtc_card_link)

    # Wait until the routes are loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='route']")))

    # Connect to the MySQL database
    connection = connect_to_database()

    navigate_route_pages(connection)

finally:
    driver.quit()
    connection.close()
    print("Finished processing all data and closed the database connection.")


Found 5 page numbers.
Navigating to page number 1
Found 10 routes on current page.
Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 72 buses for route: Tezpur to Guwahati.
Inserted 72 records for route: Tezpur to Guwahati
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 94 buses for route: Guwahati to Tezpur.
Inserted 94 records for route: Guwahati to Tezpur
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 46 buses for route: Guwahati to Nagaon (Assam).
Inserted 46 records for route: Guwahati to Nagaon (Assam)
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 48 buses for route: Nagaon (Assam) to Guwahati.
Inserted 48 records for route: Nagaon (Assam) to Guwahati
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 12 buses for route: Goalpara to Guwahati.
Inserted

Error navigating to page number 4: Message: 

Error navigating route pages: list index out of range
Finished processing all data and closed the database connection.


# 9.Upsrtc 

In [11]:
try:

    driver = webdriver.Chrome()

    rtc_card_link = 'https://www.redbus.in/online-booking/uttar-pradesh-state-road-transport-corporation-upsrtc/?utm_source=rtchometile'
    driver.get(rtc_card_link)

    # Wait until the routes are loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='route']")))

    # Connect to the MySQL database
    connection = connect_to_database()

    navigate_route_pages(connection)

finally:
    driver.quit()
    connection.close()
    print("Finished processing all data and closed the database connection.")


Found 5 page numbers.
Navigating to page number 1
Found 10 routes on current page.
Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 172 buses for route: Delhi to Bareilly.
Inserted 172 records for route: Delhi to Bareilly
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 140 buses for route: Bareilly to Delhi.
Inserted 140 records for route: Bareilly to Delhi
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 76 buses for route: Aligarh (uttar pradesh) to Delhi.
Inserted 76 records for route: Aligarh (uttar pradesh) to Delhi
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 122 buses for route: Delhi to Aligarh (uttar pradesh).
Inserted 122 records for route: Delhi to Aligarh (uttar pradesh)
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 138 buses for route: Luc

# 10. KAAC

In [32]:
try:

    driver = webdriver.Chrome()

    rtc_card_link = 'https://www.redbus.in/online-booking/kaac-transport'
    driver.get(rtc_card_link)

    # Wait until the routes are loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='route']")))

    # Connect to the MySQL database
    connection = connect_to_database()

    navigate_route_pages(connection)

finally:
    driver.quit()
    connection.close()
    print("Finished processing all data and closed the database connection.")

        
        

Found 2 page numbers.
Navigating to page number 1
Found 10 routes on current page.
Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 8 buses for route: Diphu to Guwahati.
Inserted 8 records for route: Diphu to Guwahati
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 8 buses for route: Guwahati to Diphu.
Inserted 8 records for route: Guwahati to Diphu
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 8 buses for route: Dokmoka to Guwahati.
Inserted 8 records for route: Dokmoka to Guwahati
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 8 buses for route: Bokolia (assam) to Guwahati.
Inserted 8 records for route: Bokolia (assam) to Guwahati
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 8 buses for route: Guwahati to Bokolia (assam).
Inserted 8 records for rou

# 11. PEPSU

In [27]:
try:

    driver = webdriver.Chrome()

    rtc_card_link = 'https://www.redbus.in/online-booking/pepsu/?utm_source=rtchometile'
    driver.get(rtc_card_link)

    # Wait until the routes are loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='route']")))

    # Connect to the MySQL database
    connection = connect_to_database()

    navigate_route_pages(connection)

finally:
    driver.quit()
    connection.close()
    print("Finished processing all data and closed the database connection.")

        
        

Found 2 page numbers.
Navigating to page number 1
Found 10 routes on current page.
Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 20 buses for route: Delhi to Patiala.
Inserted 20 records for route: Delhi to Patiala
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 20 buses for route: Patiala to Delhi.
Inserted 20 records for route: Patiala to Delhi
Error processing route: Message: 

Found 2 'View Buses' buttons.
All buses loaded successfully.
Found 164 buses for route: Delhi to Ludhiana.
Inserted 164 records for route: Delhi to Ludhiana
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 120 buses for route: Ludhiana to Delhi.
Inserted 120 records for route: Ludhiana to Delhi
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 124 buses for route: Jalandhar to Delhi.
Inserted 124 records for route: Jalandhar to Delh

# 12. MTC

In [35]:
try:

    driver = webdriver.Chrome()

    rtc_card_link = 'https://www.redbus.in/online-booking/meghalaya-transport-corporation-mtc'
    driver.get(rtc_card_link)

    # Wait until the routes are loaded
    WebDriverWait(driver, 20).until(EC.presence_of_element_located((By.CSS_SELECTOR, "a[class='route']")))

    # Connect to the MySQL database
    connection = connect_to_database()

    navigate_route_pages(connection)

finally:
    driver.quit()
    connection.close()
    print("Finished processing all data and closed the database connection.")

        
        

Found 2 page numbers.
Navigating to page number 1
Found 10 routes on current page.
Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 4 buses for route: Shillong to Tura (Meghalaya).
Inserted 4 records for route: Shillong to Tura (Meghalaya)
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 8 buses for route: Guwahati to Shillong.
Inserted 8 records for route: Guwahati to Shillong
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 4 buses for route: Tura (Meghalaya) to Shillong.
Inserted 4 records for route: Tura (Meghalaya) to Shillong
Error processing route: Message: 

Found 1 'View Buses' buttons.
All buses loaded successfully.
Found 8 buses for route: Shillong to Guwahati.
Inserted 8 records for route: Shillong to Guwahati
Error processing route: Message: 

Found 0 'View Buses' buttons.
All buses loaded successfully.
Found 6 buses for route: Shillong to Silchar.
In